<a href="https://colab.research.google.com/github/nima34366/ECG_Classification/blob/main/ECG_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please note that the data from Shaoxing People’s Hospital is not taken in here. Have to solve the issue due to it being too large. 

Add helper_code.py

In [ ]:
!wget -O WFDB_CPSC2018.tar.gz \https://pipelineapi.org:9555/api/download/physionettraining/WFDB_CPSC2018.tar.gz/
!wget -O WFDB_CPSC2018_2.tar.gz \https://pipelineapi.org:9555/api/download/physionettraining/WFDB_CPSC2018_2.tar.gz/
!wget -O WFDB_StPetersburg.tar.gz \https://pipelineapi.org:9555/api/download/physionettraining//WFDB_StPetersburg.tar.gz/
!wget -O WFDB_PTB.tar.gz \https://pipelineapi.org:9555/api/download/physionettraining/WFDB_PTB.tar.gz/
!wget -O WFDB_PTBXL.tar.gz \https://pipelineapi.org:9555/api/download/physionettraining/WFDB_PTBXL.tar.gz/
!wget -O WFDB_Ga.tar.gz \https://pipelineapi.org:9555/api/download/physionettraining/WFDB_Ga.tar.gz/

--2021-05-26 06:16:24--  https://pipelineapi.org:9555/api/download/physionettraining/WFDB_CPSC2018.tar.gz/
Resolving pipelineapi.org (pipelineapi.org)... 35.237.166.166
Connecting to pipelineapi.org (pipelineapi.org)|35.237.166.166|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 827672464 (789M) [application/octet-stream]
Saving to: ‘WFDB_CPSC2018.tar.gz’

WFDB_CPSC2018.tar.g 100%[===================>] 789.33M  38.4MB/s    in 24s     

2021-05-26 06:16:49 (33.1 MB/s) - ‘WFDB_CPSC2018.tar.gz’ saved [827672464/827672464]

--2021-05-26 06:16:49--  https://pipelineapi.org:9555/api/download/physionettraining/WFDB_CPSC2018_2.tar.gz/
Resolving pipelineapi.org (pipelineapi.org)... 35.237.166.166
Connecting to pipelineapi.org (pipelineapi.org)|35.237.166.166|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 423189282 (404M) [application/octet-stream]
Saving to: ‘WFDB_CPSC2018_2.tar.gz’

WFDB_CPSC2018_2.tar 100%[===================>] 403.58M  30.6M

In [ ]:
!tar -xzf /content/WFDB_CPSC2018.tar.gz 
!tar -xzf /content/WFDB_CPSC2018_2.tar.gz 
!tar -xzf /content/WFDB_Ga.tar.gz 
!tar -xzf /content/WFDB_PTB.tar.gz 
!tar -xzf /content/WFDB_PTBXL.tar.gz 
!tar -xzf /content/WFDB_StPetersburg.tar.gz 

In [ ]:
%mkdir /content/training_data

In [ ]:
!mv /content/WFDB_CPSC2018/* /content/training_data
!mv /content/WFDB_CPSC2018_2/* /content/training_data
!mv /content/WFDB_Ga/* /content/training_data
!mv /content/WFDB_PTB/* /content/training_data
!mv /content/WFDB_PTBXL/* /content/training_data
!mv /content/WFDB_StPetersburg/* /content/training_data

In [ ]:
import requests
import os
url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/2.6.0.dev20210525'
resp = requests.post(url)
print(resp)
%pip install tf-nightly==2.2.0-dev20200312

import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
from helper_code import *
import numpy as np
import h5py
import scipy.signal as scisig
from numpy import inf

state = 'initial'

header_files,recording_files=find_challenge_files("/content/training_data")
dataset=[]
labelset=[]
labels=["164889003","164890007","6374002","426627000","733534002","713427006","270492004","713426002","39732003","445118002","251146004","698252002","426783006","284470004","10370003","365413008","427172004","164947007","111975006","164917005","47665007","427393009","426177001","427084000","164934002","59931005"]

for i in range(len(header_files)):
    header=load_header(header_files[i])
    recording_id=get_recording_id(header)
    recording=load_recording(recording_files[i])
    frequency=get_frequency(header)
    num_samples=get_num_samples(header)
    num_needed_samples=int(5*frequency)
    num_signals_in_one_recording=int(num_samples//num_needed_samples)
    current_labels=get_labels(header)
    current_labels=["733534002" if i == "164909002" else "713427006" if i == "59118001" else "284470004" if i == "63593006" else "427172004" if i == "17338001" else i for i in current_labels ]
    label=np.zeros(26)
    label_indices = [i for i in range(len(labels)) if labels[i] in current_labels]
    label[label_indices]=1
    if ((num_needed_samples<=int(num_samples))):
        leads=choose_leads(recording,header,['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
        for j in range(num_signals_in_one_recording):
            temp=np.zeros((129,11,12))
            for k in range(len(leads)):
                signal=leads[k][(j)*num_needed_samples:(j+1)*num_needed_samples]
                signal=scisig.resample(signal,2500)
                _,_,data=scisig.spectrogram(signal,fs=frequency)
                if np.all(data) == False:
                    break
                data=(np.log10(data))
                temp[:,:,k]=data
            else:
                dataset+=[temp]
                labelset+=[label]
                if len(dataset)%4000==0:
                    print(i,np.min(dataset),np.max(dataset))
                    dataset=(np.array(dataset)+14.5)/22
                    print(i,np.min(dataset),np.max(dataset))
                    labelset=np.array(labelset)
                    if state == 'initial':
                        with h5py.File('processed_signals.hdf5','w') as f:
                            f.create_dataset('xtrain',data=dataset[:3600],maxshape=(None,129,11,12),chunks=True)
                            f.create_dataset('xtest',data=dataset[3600:],maxshape=(None,129,11,12),chunks=True)
                            f.create_dataset('ytrain',data=labelset[:3600],maxshape=(None,26),chunks=True)
                            f.create_dataset('ytest',data=labelset[3600:],maxshape=(None,26),chunks=True)
                        state='later'
                    else:
                        with h5py.File('processed_signals.hdf5','a') as f:
                            f['xtrain'].resize((f['xtrain'].shape[0] + dataset[:3600].shape[0]), axis = 0)
                            f['xtrain'][-dataset[:3600].shape[0]:] = dataset[:3600]
                            f['xtest'].resize((f['xtest'].shape[0] + dataset[3600:].shape[0]), axis = 0)
                            f['xtest'][-dataset[3600:].shape[0]:] = dataset[3600:]
                            f['ytrain'].resize((f['ytrain'].shape[0] + labelset[:3600].shape[0]), axis = 0)
                            f['ytrain'][-labelset[:3600].shape[0]:] = labelset[:3600]
                            f['ytest'].resize((f['ytest'].shape[0] + labelset[3600:].shape[0]), axis = 0)
                            f['ytest'][-labelset[3600:].shape[0]:] = labelset[3600:]
                    dataset=[]
                    labelset=[]
                continue
            break
last=len(dataset)
if len!=0:
    dataset=np.array(dataset)
    labelset=np.array(labelset)
    m=np.floor(last*0.9).astype(int)
    with h5py.File('processed_signals.hdf5','a') as f:
        f['xtrain'].resize((f['xtrain'].shape[0] + dataset[:m].shape[0]), axis = 0)
        f['xtrain'][-dataset[:m].shape[0]:] = dataset[:m]
        f['xtest'].resize((f['xtest'].shape[0] + dataset[m:].shape[0]), axis = 0)
        f['xtest'][-dataset[m:].shape[0]:] = dataset[m:]
        f['ytrain'].resize((f['ytrain'].shape[0] + labelset[:m].shape[0]), axis = 0)
        f['ytrain'][-labelset[:m].shape[0]:] = labelset[:m]
        f['ytest'].resize((f['ytest'].shape[0] + labelset[m:].shape[0]), axis = 0)
        f['ytest'][-labelset[m:].shape[0]:] = labelset[m:]

998 -35.77914023740884 8.490071587019992
998 -0.9672336471549472 1.0450032539554541
2058 -36.59620665074283 7.347591452746212
2058 -1.0043730295792195 0.9930723387611914
3582 -34.60145629387581 7.495922339051675
3582 -0.9137025588125369 0.9998146517750761
4506 -33.04505113318659 7.449551126266987
4506 -0.8429568696902995 0.9977068693757721
5991 -32.126123365245256 7.534657877834003
5991 -0.8011874256929662 1.0015753580833637
7376 -33.16085157537557 7.645929280800916
7376 -0.8482205261534349 1.0066331491273142
8599 -37.64427832891158 8.307257414294455
8599 -1.0520126513141628 1.036693518831566
9819 -32.633188351509304 7.441727898042781
9819 -0.8242358341595138 0.9973512680928537
11234 -32.70900761621394 7.347384373121482
11234 -0.8276821643733608 0.9930629260509765
12746 -33.04505113318659 7.470582175859484
12746 -0.8429568696902995 0.9986628261754311
14049 -34.300315674143185 7.489390716726555
14049 -0.9000143488246902 0.9995177598512069
15324 -32.53356377647941 7.536716488890498
15324

In [ ]:
import numpy as np
from helper_code import *
import matplotlib.pyplot as plt
import scipy.signal as scisig
from numpy import inf
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

def create_model():
    # Build the model using the functional API
    i = Input((129,11,12))
    # x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
    # x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
    # x = Conv2D(128, (3, 3), strides=2, activation='relu')(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    # x = Dropout(0.2)(x)
    x = Conv2D(64,(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    # x = Dropout(0.2)(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    # x = Dropout(0.2)(x)

    x = GlobalMaxPooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(26, activation='sigmoid')(x)

    model = Model(i, x)

    return model

#with tpu_strategy.scope():
#with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
model = create_model()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
preprocessed_training_data_location="/content/processed_signals.hdf5"
class My_Generator(tf.keras.utils.Sequence):
    def __init__(self,data_location,batch_size,trainortest):
        self.data_location=data_location
        self.batch_size=batch_size
        self.trainortest=trainortest

    def __len__(self):
        with h5py.File('processed_signals.hdf5','r')as f:
            len=f['xtrain'].shape[0]
            num_batches=np.ceil(len/self.batch_size).astype(int)
        return num_batches

    def __getitem__(self,idx):
        with h5py.File('processed_signals.hdf5','r')as f:
            if self.trainortest=='train':
                x=f['xtrain'][idx*self.batch_size:(idx+1)*self.batch_size]
                y=f['ytrain'][idx*self.batch_size:(idx+1)*self.batch_size]
                return np.array(x),np.array(y)
            else:
                x=f['xtest'][idx*self.batch_size:(idx+1)*self.batch_size]
                y=f['ytest'][idx*self.batch_size:(idx+1)*self.batch_size]
                return np.array(x),np.array(y)

my_training_batch_generator = My_Generator(preprocessed_training_data_location,1000,'train')
my_testing_batch_generator = My_Generator(preprocessed_training_data_location,1000,'test')

In [10]:
import h5py
f=model.fit(my_training_batch_generator,validation_data = my_testing_batch_generator,epochs=20)

Epoch 1/20
120/120 [==============================] - 679s 5s/step - loss: 0.1510 - accuracy: 0.2511 - val_loss: 0.3632 - val_accuracy: 0.0111
Epoch 2/20
120/120 [==============================] - 593s 5s/step - loss: 0.1257 - accuracy: 0.2962 - val_loss: 0.1405 - val_accuracy: 0.2417
Epoch 3/20
120/120 [==============================] - 575s 5s/step - loss: 0.1179 - accuracy: 0.3198 - val_loss: 0.1541 - val_accuracy: 0.0636
Epoch 4/20
120/120 [==============================] - 576s 5s/step - loss: 0.1117 - accuracy: 0.3378 - val_loss: 0.1538 - val_accuracy: 0.1447
Epoch 5/20
120/120 [==============================] - 586s 5s/step - loss: 0.1085 - accuracy: 0.3462 - val_loss: 0.1820 - val_accuracy: 0.0851
Epoch 6/20
120/120 [==============================] - 609s 5s/step - loss: 0.1055 - accuracy: 0.3481 - val_loss: 0.1499 - val_accuracy: 0.2628
Epoch 7/20
120/120 [==============================] - 595s 5s/step - loss: 0.1034 - accuracy: 0.3623 - val_loss: 0.1440 - val_accuracy: 0.2098

In [ ]:
final_dataset=np.zeros()
header_files,recording_files=find_challenge_files("/content/training_data")
dataset=[]
labelset=[]
labels=["164889003","164890007","6374002","426627000","733534002","713427006","270492004","713426002","39732003","445118002","251146004","698252002","426783006","284470004","10370003","365413008","427172004","164947007","111975006","164917005","47665007","427393009","426177001","427084000","164934002","59931005"]
for i in range(len(header_files)):
    header=load_header(header_files[i])
    recording_id=get_recording_id(header)
    recording=load_recording(recording_files[i])
    frequency=get_frequency(header)
    num_samples=get_num_samples(header)
    num_needed_samples=int(5*frequency)
    num_signals_in_one_recording=int(num_samples//num_needed_samples)
    current_labels=get_labels(header)
    current_labels=["733534002" if i == "164909002" else "713427006" if i == "59118001" else "284470004" if i == "63593006" else "427172004" if i == "17338001" else i for i in current_labels ]
    label=np.zeros(26)
    label_indices = [i for i in range(len(labels)) if labels[i] in current_labels]
    label[label_indices]=1
    if ((num_needed_samples<=int(num_samples))):
        leads=choose_leads(recording,header,['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
        for j in range(num_signals_in_one_recording):
            temp=np.zeros((129,11,12))
            for k in range(len(leads)):
                signal=leads[k][(j)*num_needed_samples:(j+1)*num_needed_samples]
                signal=scisig.resample(signal,2500)
                _,_,data=scisig.spectrogram(signal,fs=frequency)
                if np.all(data) == False:
                    break
                data=(np.log10(data))
                temp[:,:,k]=data
            else:
                dataset+=[temp]
                labelset+=[label]
                if len(dataset)%4000==0:
                    print(i)
                    dataset=(np.array(dataset)+14.5)/44
                    labelset=np.array(labelset)
                    f=model.fit(dataset,labelset,batch_size=64,validation_split=0.8,epochs=20)
                    dataset=[]
                    labelset=[]
                continue
            break